In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from tqdm import tqdm
from scipy import ndimage

In [2]:
import torch.nn.functional as F
import torch as tc
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from helpers_img import *
from NeuralNets import *
from training_NN import *
from preprocessing import *

# Valerio inizia da qui

In [3]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
N = min(100, len(files)) # Load maximum 100 images
print("Loading " + str(N) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(N)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(N) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(N)]
print(files[0])

#n = 85 # Only use 85 images for training

Loading 100 images
satImage_042.png
Loading 100 images
satImage_042.png


In [ ]:
class DatasetUNet(tc.utils.data.Dataset):
    def __init__(self, root_dir):
        self.image_dir + root_dir + "images/"
        self.files = os.listdir(self.image_dir)
        self.gt_dir = root_dir + "groundtruth/"
    
    def __len__(self):
        return len(self.files)
    
    def 

In [ ]:
dataset,label = rotation(imgs, gt_imgs)
dataset = preprocessing_for_UNet(dataset)
dataset,label = flip(dataset,label)
dataset = from_list_to_tensor(dataset)
label = from_list_to_tensor(dataset)

dataset = tc.utils.data.TensorDataset(dataset,label)
dataset = tc.utils.data.DataLoader(dataset)

In [ ]:
image = imgs[0]
plt.figure(figsize=(10,10))
plt.imshow(image)


In [ ]:
mean = image.mean()
std = image.std()

noise = np.random.normal(mean,std/5,size = image.shape)
image_noise = image + noise
plt.figure(figsize=(10,10))
plt.imshow(image_noise)

In [4]:
# augment the dataset adding rotated images
imgs, gt_imgs = rotation(imgs, gt_imgs)
print('Total number of imgages: '+str(len(imgs)))

Total number of imgages: 400


# The U-Net

In [5]:
class UNet(tc.nn.Module):
    def __init__(self,features=3):
        super(UNet,self).__init__()
        self.is_training=False
        self.conv1 = tc.nn.Conv2d(features,8,kernel_size=3)
        self.batch1 = tc.nn.BatchNorm2d(8)
        self.conv2 = tc.nn.Conv2d(8,8, kernel_size=3)
        self.batch2 = tc.nn.BatchNorm2d(8)
        self.conv3 = tc.nn.Conv2d(8,16, kernel_size=3)
        self.batch3 = tc.nn.BatchNorm2d(16)
        self.conv4 = tc.nn.Conv2d(16,16, kernel_size=3)
        self.batch4 = tc.nn.BatchNorm2d(16)
        self.conv5 = tc.nn.Conv2d(16,16, kernel_size=4)
        self.batch5 = tc.nn.BatchNorm2d(16)
        self.convUp1 = tc.nn.ConvTranspose2d(16,16,stride=2,kernel_size=2)
        self.conv6 = tc.nn.Conv2d(32,16, kernel_size=3)
        self.batch6 = tc.nn.BatchNorm2d(16)
        self.conv7 = tc.nn.Conv2d(16,16, kernel_size=3)
        self.batch7 = tc.nn.BatchNorm2d(16)
        self.convUp2 = tc.nn.ConvTranspose2d(16,8,stride=2,kernel_size=2)
        self.conv8 = tc.nn.Conv2d(16,8, kernel_size=3)
        self.batch8 = tc.nn.BatchNorm2d(8)
        self.conv9 = tc.nn.Conv2d(8,8, kernel_size=3)
        self.batch9 = tc.nn.BatchNorm2d(8)
        self.convUp3 = tc.nn.ConvTranspose2d(8,4,kernel_size=39)
        self.conv10 = tc.nn.Conv2d(4,1,kernel_size=3)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.batch1(x)
        x = F.relu(self.conv2(x))
        #x1 = x.clone()
        x1 = x
        x = F.max_pool2d(x, kernel_size=2)
        x = x = self.batch2(x)
        x = F.relu(self.conv3(x))
        x = self.batch3(x)
        x = F.relu(self.conv4(x))
        #x2 = x.clone()
        x2 = x
        x = F.max_pool2d(x, kernel_size=2)
        x = self.batch4(x)
        x = F.relu(self.conv5(x))
        x = self.batch5(x)
        
        x = F.relu(self.convUp1(x))
        
        x= self.adapt_activation(x2,x)
        
        x = F.relu(self.conv6(x))
        x = self.batch6(x)
        x = F.relu(self.conv7(x))
        x = self.batch7(x)
        
        x = F.relu(self.convUp2(x))
        
        x = self.adapt_activation(x1,x)
        
        x = F.relu(self.conv8(x))
        x = self.batch8(x)
        x = F.relu(self.conv9(x))
        x = self.batch9(x)
        
        x = F.relu(self.convUp3(x))
        x = tc.sigmoid(self.conv10(x))
        
        return x
    
    def adapt_activation(self,x_old,x_new):
        '''the function adapts the activation to the smaller one dimensions
        and concatenates them'''
        
        space = (x_old.size(2)- x_new.size(2))//2
        
        distance = x_new.size(2)
        
        x_old = x_old.narrow(2,space,distance)
        x_old = x_old.narrow(3,space,distance)
        
        x_new = tc.cat((x_old,x_new),dim=1)
        
        return x_new
        
        
        

In [6]:
def preprocessing_for_UNet(dataset):
    # laplace
    
    for i,image in enumerate(dataset):
        _, laplacian_image = add_laplacian(image)
        sobel = add_sobel(image)
        segment = add_segment(image)
        dataset[i]=np.concatenate((image,laplacian_image,sobel,segment),axis = 2)
    
    return dataset

In [7]:
def add_noise(dataset,label, is_simple=True):
    '''Add noise to the dataset.
    
    dataset : tensor type
    
    label : tensor type'''
    
    
    if is_simple:
        
        mean, std = dataset.mean(), dataset.std()
        # the noise has the 20% of the image standard deviation
        noise = np.random.normal(loc = mean, scale = std/5, size = dataset.size())
        

        dataset_with_noise = dataset + tc.tensor(noise).type(tc.FloatTensor)
        dataset = tc.cat((dataset,dataset_with_noise),dim = 0)
        label = label.type(tc.FloatTensor)
        label = tc.cat((label, label), dim = 0)
    else:
        
        mean, std = 0, 0.05
        
        noise = np.random.normal(mean, std, size = label.size())
        
        label.type(float)
        
        label_with_noise = label + tc.tensor(noise).type(tc.FloatTensor)
        
        label = tc.cat((label,label_with_noise),dim=0)
        
        dataset = tc.cat((dataset,dataset),dim=0)
    
    return dataset, label
        
        
    
    

In [8]:
def from_list_to_tensor(dataset):
    ''' cast a list of image in a tensor of appropriate size'''
    dataset = np.array(dataset)
    try :
        N,rows,columns,features = dataset.shape
    except:
        N,rows,columns = dataset.shape
        features=1
        dataset=dataset.reshape(N,rows,columns,features)
        
    dataset_tensor = tc.Tensor(N, features, rows, columns)
    for j in range(N):
        dataset_tensor[j] = tc.tensor(np.array([dataset[j,:,:,i] for i in range(features)]))
    
    
    return dataset_tensor

In [9]:
def train_UNet(dataset, label, lr, max_epochs, mini_batch_size, nb_test, threshold=0.5, 
               do_preprocessing = False, flip_data=True, model=None):
    ''' train the UNet using the Binary Cross Entropy loss and Adam as optimizer.
    The dataset must be a list of 400*400 images.'''
    
    if flip_data: 
        dataset,label = flip(dataset,label)
    
    if do_preprocessing:
        dataset = preprocessing_for_UNet(dataset)
    
    
    dataset = from_list_to_tensor(dataset)
    mean = dataset.mean()
    std = dataset.std()
    dataset = (dataset - mean)/ std
    test= dataset.narrow(0,dataset.size(0)-nb_test,nb_test)
    dataset = dataset.narrow(0,0,dataset.size(0)-nb_test)
    label = from_list_to_tensor(label)
    test_l = label.narrow(0,dataset.size(0),nb_test)
    label = label.narrow(0,0,dataset.size(0))
    
    ## add is_simple = False to add noise to labels
    dataset, label = add_noise(dataset, label)
    if model == None:
        model = UNet(features= dataset.size(1))
    
    optimizer=tc.optim.Adam(model.parameters(),lr)
    # maybe using MSE is better
    #criterion= tc.nn.BCELoss()
    criterion = tc.nn.MSELoss()
    training_errors=[]
    losses=[]
    
    if tc.cuda.is_available():
        print('cuda is available')
        model.cuda()
        #criterion.cuda()
        test = test.cuda()
        #dataset= dataset.cuda()
        #label = label.cuda()
    
    #training_F1_error=[]
    #print('starting to train the net')
    for epoch in tqdm(range(max_epochs)):
        model.train()
        #if tc.cuda.is_available():
        #    model.cuda()
        for j in range(0,dataset.size(0)-nb_test,mini_batch_size):
            input_data = dataset.narrow(0,j,mini_batch_size)
            label_data = label.narrow(0,j,mini_batch_size)
            if tc.cuda.is_available():
                input_data, label_data = input_data.cuda(), label_data.cuda()
            output = model(input_data)
            #print(output, label_data)
            loss = criterion(output, label_data)
            model.zero_grad()
            loss.backward()
            optimizer.step()
            tc.cuda.empty_cache()
            
        
        # compute training error
        model.eval()
        losses.append(loss)
        prediction = model(test)
        prediction = prediction.cpu()
        prediction = prediction.detach_().numpy()[:,0,:,:]
        prediction = (prediction > threshold)*1
        #print(prediction[0])
        mask = test_l.numpy()[:,0,:,:]
        F1_error = 0
        #print(mask.shape)
        training_error = (((mask>0.5)*1 == prediction)*1).sum()/np.prod(prediction.shape)
        
        training_errors.append(training_error)
        
        
        # improve the calcul_F1 fonction to optimize the code
        '''
        for i in range(dataset.size(0)):
            F1_error += calcul_F1(mask[i], prediction[i])
        F1_error = F1_error/dataset.size(0)
        training_F1_error.append(F1_error)
        if F1_error >0.95:
            break
        
    # plot the results
    
    plt.figure()
    plt.plot(np.arange(epoch + 1)+1,training_F1_error)
    plt.xlabel('epoch')
    plt.ylabel('F1_error')
    '''
    plt.figure()
    plt.plot(np.arange(epoch + 1)+1,losses)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.figure()
    plt.plot(np.arange(epoch + 1)+1,training_errors)
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    return model
        
    

In [ ]:
from torchvision import transforms, datasets

In [ ]:
? datasets.ImageFolder

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset


In [ ]:
print(tc.cuda.is_available())

In [ ]:
tc.cuda.empty_cache()

In [11]:
N = len(imgs)
n = N
train_imgs=imgs[0:n]
train_mask=gt_imgs[0:n]
#test_imgs=imgs[n:N]
#test_mask = gt_imgs[n:N]
lr = 1e-4
max_epochs = 50
mini_batch_size=10
nb_test = 10
print('let\'s start to train the Net')

model = train_UNet(train_imgs, train_mask, lr, max_epochs, mini_batch_size,nb_test, do_preprocessing=True, model=model)


let's start to train the Net


RuntimeError: $ Torch: not enough memory: you tried to allocate 4GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1544081127912/work/aten/src/TH/THGeneral.cpp:201

In [ ]:
model.cpu()

In [ ]:
model.cpu()
tc.save(model,'Model_UNet/model_CPU.pt')

In [10]:
model = tc.load('Model_UNet/model_CPU.pt')

In [ ]:
# test on a image
image = [imgs[4]]
image = preprocessing_for_UNet(image)
image = from_list_to_tensor(image)
image = (image - image.mean())/image.std()
model.eval()
prediction = model(image)
print(prediction.size())
prediction =  prediction.detach_().numpy()[0,0,:,:]
prediction = (prediction > 0.5)*1
#mask = gt_imgs[20]
mask = prediction
image = make_img_overlay(imgs[4],mask)

plt.figure(figsize=(10,10))
plt.imshow(image)







In [ ]:
def create_submission_UNet(test_set, model, name_submission, do_preprocessing= False):
    ''' create the submission file for U-Net.
    
    test_set is a list of images'''
    
    # preprocessing
    
    if do_preprocessing:
        test_set = preprocessing_for_UNet(test_set)
        
    # transform list of images in tensor 
    print('a')
    test_set = from_list_to_tensor(test_set)
    print('a')
    test_set = (test_set- test_set.mean())/test_set.std()
    model.eval()
    
    mini_batch = 10
    print('a')
    N,channel,w,h = test_set.size()
    if tc.cuda.is_available():
        model.cuda()
    
    mask = tc.Tensor(N,1,w,h)
    batch_size=10
    for i in range(0,test_set.size(0),mini_batch):
        temp =  test_set.narrow(0,i,mini_batch)
        print(temp.size())
        if tc.cuda.is_available():
            tc.cuda.empty_cache()
            temp = temp.cuda()
        temp = model(temp)
        mask[i:i+batch_size] = temp.cpu()
    print('a')
    mask = (mask.detach().numpy() > 0.5)*1
    list_of_names= []
    for i in range(N):
        plt.imsave( "prediction_"+str(i+1)+".png", mask[i], cmap = matplotlib.cm.gray)
        list_of_names.append("prediction_"+str(i+1)+".png")
    masks_to_submission(name_submission, *list_of_names)

In [ ]:
# Loaded a set of images
root_dir_test = "test_set_images/"
imgs_test = []
for l in range(1,51):
    dir_test = root_dir_test + 'test_'+str(l)+'/'
    files_test = os.listdir(dir_test)
    img_test = load_image(dir_test + files_test[0])
    imgs_test.append(img_test)

In [ ]:
create_submission_UNet(imgs_test, model, 'third_submission.csv', do_preprocessing= True)

In [ ]:
n=90*3
N=len(imgs)
train_imgs=imgs[0:n]
train_mask=gt_imgs[0:n]
test_imgs=imgs[n:N]
test_mask = gt_imgs[n:N]


In [ ]:
temp=concatenate_images(train_imgs[3], train_mask[3])
plt.figure(figsize=(10, 10))
plt.imshow(temp)

In [ ]:
? tc.nn.Dropout

In [ ]:
print(len(train_mask))

In [ ]:
print(len(test_imgs),N-n)

In [ ]:
# prepare input data 
w=16
h=16
train_sub_images=[img_crop(train_imgs[i], w, h) for i in range(n)]
train_mask_label=[img_crop(train_mask[i],w,h) for i in range(n)]
train_mask_label=from_mask_to_vector(train_mask_label,0.3)
test_sub_images=[img_crop(test_imgs[i], w, h) for i in range(N-n)]
#test_mask_label=[img_crop(train_mask[n+i],w,h) for i in range(n)]
#test_mask_label=from_mask_to_vector(test_mask_label)


In [ ]:
? Image.fromarray

In [ ]:
print((1*train_mask_label).sum()/len(train_mask_label))

In [ ]:
print(len(train_sub_images[0]))

In [ ]:
train_sub_images = transform_subIMG_to_Tensor(train_sub_images)
test_sub_images = transform_subIMG_to_Tensor(test_sub_images)

In [ ]:
# normalize data
mean = train_sub_images.mean()
std = train_sub_images.std()
train_sub_images = (train_sub_images-mean)/std
train_sub_images, train_mask_label = reduce_dataset(train_sub_images,train_mask_label)
#print(train_sub_images[-1]-train_sub_images[-2])
for l in range(n):
    new_indices= np.random.permutation(len(train_mask_label))
    train_sub_images=train_sub_images[new_indices]
    train_mask_label=train_mask_label[new_indices]
#print(train_sub_images[0]-train_sub_images[1])
#train_sub_images.requires_grad_(True)

In [ ]:
 ? tc.Tensor.narrow

In [ ]:
print(train_mask_label.sum()/len(train_mask_label))

In [ ]:
model = SimpleNet(0)
lr=1e-4
max_epochs=15
mini_batch_size=1

train_model_Adam( model, train_sub_images, train_mask_label, max_epochs, lr, mini_batch_size)

In [ ]:
print(tc.__version__)

In [ ]:
? tc.mean

In [ ]:
N1=625

In [ ]:
# print the result on a single image
#N1=int(train_sub_images.size(0)/n)
image_test= test_sub_images.narrow(0,43*N1,N1)
image_test=(image_test-mean)/std
mask_array=model(image_test)
#print(mask_array.max())
mask_list=mask_array[:]>0.5
mask_test = label_to_img(400, 400, 16, 16, mask_list)

In [ ]:
# compute F1 score
mask=gt_imgs[n]
#print(mask.sum())
print(calcul_F1(mask, mask_test))

In [ ]:
# overlap images

print_img = make_img_overlay(imgs[n+43], mask_test)
#print_img = make_img_overlay(imgs[n+43], gt_imgs[n+43])
plt.figure(figsize=(10, 10))
plt.imshow(print_img)

In [ ]:
def compute_test_error(test_dataset, test_label, nb_patches, model):
    ''' compute the F1 error made during road segmentation. 
    Exemple: for a 16*16 patch we have that nb_patches=625'''
    model.is_train=False
    mean=test_dataset.mean()
    std= test_dataset.std()
    test_dataset = (test_dataset-mean)/std
    test_output = model(test_dataset)
    w = int(400/np.sqrt(nb_patches))
    F1_error=np.zeros(int(test_dataset.size(0)/nb_patches))
    for i in range(int(test_dataset.size(0)/nb_patches)):
        mask = test_output.narrow(0,i*nb_patches, nb_patches)
        mask = mask.detach().numpy()[:]>0.5
        mask = label_to_img(400, 400, w, w, mask)
        F1_error[i] = calcul_F1(test_label[i], mask)
    plt.figure()
    plt.plot(np.arange(int(test_dataset.size(0)/nb_patches))+1,F1_error)
    plt.xlabel('image')
    plt.ylabel('F1_error')
    F1_mean= np.mean(F1_error)
    return F1_mean
    

In [ ]:
F1=compute_test_error(test_sub_images, test_mask, 625, model)

In [ ]:
print(F1)